In [14]:
import nltk
import pandas as pd
from bs4 import BeautifulSoup
import json
import lxml
import pandas as pd


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
tokenizer = nltk.tokenize.PunktTokenizer('english')

In [12]:
input_file = "test_ans.json"
query_file = "topics_2.json"

with open(input_file,'r') as file:
    answers = json.load(file)

with open(query_file,'r') as file:
    querys = json.load(file)

In [10]:
def process_input(data):
    for item in data:
        text = item['Text']
        soup = BeautifulSoup(text, "lxml")
        text = soup.get_text()
        item['Text'] = text

In [13]:
process_input(answers)


In [ ]:
#answers is in the format of Id: '1234', Text: 'text goes here', score:'12'